# Creating a simple blockchain

## Project from "Special Topics on Computer Systems"


Following the tutorial available at: https://levelup.gitconnected.com/creating-a-blockchain-from-scratch-9a7b123e1f3e

To understand the overall behavior of a simple blockchain, consider the blockchain as a database. For instance, we can think that in a:
- Form: we want to store some data we are providing and retrieve them when needed;
- Blog: like the form, we want the data to be provided, stored and retrieved when desired;
- Store: as a transaction medium, we want to share data (the product itself, its available amount and price) and want to retrieve the buyer's payment and address to send the product;

With that said, consider the blocks (the components of a blockchain) as container objects we directly manage: the data provided (product ID, availability and price, if considering a store) and a timestamp, for instance. Lastly, it must contain the cryptographic label of the block it will be attached to in order to assure security.

In [2]:
from datetime import datetime
import hashlib

In [30]:
# Initialize a block to place within the blockchain

# The created block must have:
#     data (to be stored within the decentralized chain network)
#     previous_block_hash (to be attached to to grant security)

class Block():
    def __init__(self, data, previous_block_hash):
        
        # reserve the time stamp when the block is required to be created
        self.timestamp = datetime.utcnow()
        
        # Store the data put inside the block
        self.data = data
        
        # Store the encoded cryptographic label from previous block
        self.prev_block_hash = previous_block_hash
        
        # Calculate the cryptography for the current block
        self.hash = hashlib.sha256(self.to_string().encode()).hexdigest()
        
        # Recalculate the cryptography to Proof-of-Work (PoW)
        self.calculate_valid_hash()
        
    def to_string(self):
        
        # Return the block specificity:string containing data + timestamp + prev. encryptation
        return "{}\t{}\t{}".format(self.data, self.timestamp, self.prev_block_hash)
    
    def is_hash_valid(self, _hash):
        
        # Return if the recalculated encryptation is valid (PoW)
        return (_hash.startswith('0'*3))
    
    def calculate_valid_hash(self):
        
        # Initialize a empty string as a encrypted label
        _hash = ''
        
        # Initialize a bit-like disturbance over the created cryptography
        nonce = 0
        
        while (not self.is_hash_valid(_hash)):
            
            # Store a temporary variable with the block specific string 
            # (data + timestamp + prev. encryptation) and add the bit-like disturbance
            temp = self.to_string() + str(nonce)
            #print(temp)
            
            # Recalculate the cryptography considering the bit-like disturbance
            _hash = hashlib.sha256(temp.encode()).hexdigest()
            
            # Increment bit-like the disturbance
            nonce += 1
            
        # Once done, store the current block's cryptographic label 
        self.hash = _hash
        
        # Store the nonce as static to be easily accessed
        self.nonce = nonce

In [31]:
# Create a blockchain. The network enlacing the uploaded blocks
# For each block, the blockchain stores:
#     data, timestamp, cryptographic label

class Blockchain():
    def __init__(self):
        
        # Initialize the blockchain as a array
        self.blocks = []
        
        # Place the first block inside the blockchain
        self.set_genesis_block()
        
    def set_genesis_block(self):
        
        # Generate the data from the first block
        data = 'Genesis'
        
        # Generate the first cryptogtaphic label (trivial)
        prev_hash = '0'*64
        
        # Generate a block container with: data + prev. encryptation
        genesis_block = Block(data, prev_hash)
        
        # Add the block to the blockchain
        self.blocks.append(genesis_block)
        
    def get_last_hash(self):
        
        # Get the last block placed in the blockchain
        last_block = self.blocks[-1]
        
        # Get the last encryption label
        last_hash = last_block.hash
        
        # Return the last encrpytion label
        return (last_hash)
    
    def add_new_block(self, data):
        
        # Get the last encryption label (from the last block added to the blockchain)
        prev_hash = self.get_last_hash()
        
        # Generate the block with the provided date
        new_block = Block(data, prev_hash)
        
        # Add the new block to the blockchain
        self.blocks.append(new_block)
        

## Understanding the Blockchain functioning

In [32]:
blockchain = Blockchain()

When we start our blockchain, we create a list containing one block, our 'Genesis' block. It contains the following:

In [33]:
blocks = blockchain.blocks
print('Blockchain:\n\n', blocks[0].to_string())

Blockchain:

 Genesis	2021-05-06 06:24:48.863076	0000000000000000000000000000000000000000000000000000000000000000


Those data represents the 'Data' the block contains, the 'Timestamp' the block was created and the 'Cryptography label' for the first added block. After that, we can added other blocks

In [34]:
blockchain.add_new_block('First block')
blockchain.add_new_block('Second block')

blocks = blockchain.blocks
print('Blockchain:\n\n', blocks[0].to_string(), '\n', 
                         blocks[1].to_string(), '\n', 
                         blocks[2].to_string())

Blockchain:

 Genesis	2021-05-06 06:24:48.863076	0000000000000000000000000000000000000000000000000000000000000000 
 First block	2021-05-06 06:24:50.752372	000c0e3247452e0919f4ad8746a559baf5a8ab7508f7e2cc4b22bc1842c8894d 
 Second block	2021-05-06 06:24:50.754882	000a553cabbb8bcf3de447de6997047ef0be28c610523a7d58ac12e5c6c78f4b


On a blockchain with multiple users, simultaneous additions could be make and a multi-connected block would exist. However, here we are adding the blocks sequentially. Therefore 

- block 1 is created connected to block 0; 
- block 2 is created connected to block 1; 
- block 3 is created connected to block 2; 
- and so on.

to perform the connection, the first block's encryption must depends on the cryptographic label from the 'Genesis' block. The link is created as follow:

1. define a cryptographic function that will be used by the Blockchain. Here we use sha256;
2. the function must convert some input string into some encoded string. Here the encoded is a string with 64 spaces for {0-9, a-z} combination;
3. the input string must contain at least the previous block's cryptographic label. Here we combine data + timestamp + cryptographic label and use this new string to generate the encoded label for the new added blocimpor
4. to encourage the blockchain integrity, a consensus protocol must be added. Here we use the Proof-of-Work (PoW) algorithm. It makes the block creation a dispendious task, though rewarded

The Proof-of-Work (PoW) consensus protocol used here add an incremental number to the end of the input string and only accept the new block if the generated cryptographic label starts with '000'.

We know that

In [35]:
print(blocks[0].to_string())

Genesis	2021-05-06 06:24:48.863076	0000000000000000000000000000000000000000000000000000000000000000


are the three elements we use to encode the next block's label. So we expect that using these elements alone suffices to generate the 'First block' cryptographic label. We can call it 'raw encoded label'

In [56]:
encoded_label = hashlib.sha256(blocks[0].to_string().encode()).hexdigest()
print('Raw encoded label for the First block:\n\n', encoded_label)

Raw encoded label for the First block:

 bb880e63eddb29ea178b58785b98cf568828256447d1dc08c4a3106d7c57ab80


However we can see that the real encoded label is different

In [39]:
print(blocks[1].to_string())

First block	2021-05-06 06:24:50.752372	000c0e3247452e0919f4ad8746a559baf5a8ab7508f7e2cc4b22bc1842c8894d


In order to generate the Proof-of-Work (PoW), we actually need a 'nonce', which is the number we add to the end of the string to validate the new block.

We can get the nonce from the 'Genesis' block 

In [46]:
print("Genesis block:\n\n",'Nonce:', blocks[0].nonce)

Genesis block:

 Nonce: 533


In [53]:
nonced_string = blocks[0].to_string() + str(blocks[0].nonce - 1) # The -1 is considered because the validation occurs
                                                                 # after the unity increment
print('True string to encoded for the first block added:\n\n', nonced_string)

encoded_label = hashlib.sha256(nonced_string.encode()).hexdigest()
print('True encoded label for the First block:\n\n', encoded_label)

True string to encoded for the first block added:

 Genesis	2021-05-06 06:24:48.863076	0000000000000000000000000000000000000000000000000000000000000000532
True encoded label for the First block:

 000c0e3247452e0919f4ad8746a559baf5a8ab7508f7e2cc4b22bc1842c8894d


And we can see that this new encrypted label is the real label we have for the valid 'First block'

In [55]:
print(blocks[1].to_string())

First block	2021-05-06 06:24:50.752372	000c0e3247452e0919f4ad8746a559baf5a8ab7508f7e2cc4b22bc1842c8894d


With the Proof-of-Work (PoW), new blocks are laboriously added, but this action is rewarded for miners (how spend money to mine blocks, earn money from the blockchain structure. For instance bitcoin) and the connections between blocks are strengthen and the data put into the blockchain is more valid for auditorship.